In [4]:
import pandas as pd
from itertools import product
from src.ranking import calculate_ranking
from src.simulation import simulate_round_all

ModuleNotFoundError: No module named 'ranking'

In [ ]:
def calculate_round5_probabilities(df, round_num):
    """
    Calculate the probability for each team to need to play in Round 5 after Round 4.

    Args:
    - df: DataFrame containing match results up to Round 3.
    - round_num: The current round number (should be 4).

    Returns:
    - A DataFrame with teams and their probabilities to play in Round 5.
    """
    # Get current standings after Round 3
    current_ranking = calculate_ranking(df, round_num - 1)
    teams = current_ranking["Team"].tolist()
    team_records = current_ranking.set_index("Team")[["Wins", "Losses"]].to_dict("index")
    
    # Get Round 4 matchups
    matchups = round4_df[["T1", "T2"]].values.tolist()
    
    # Total possible outcomes
    total_outcomes = 2 ** len(matchups)
    
    # Initialize a dictionary to count the number of times each team needs to play in Round 5
    round5_counts = {team: 0 for team in teams}
    
    # Generate all possible outcomes
    outcome_list = list(product([0, 1], repeat=len(matchups)))
    
    for outcome in outcome_list:
        # Copy team records for this scenario
        scenario_records = {team: rec.copy() for team, rec in team_records.items()}
        
        # Apply the outcomes to the matchups
        for idx, result in enumerate(outcome):
            t1, t2 = matchups[idx]
            # If result == 1, t1 wins; else t2 wins
            if result == 1:
                scenario_records[t1]["Wins"] += 1
                scenario_records[t2]["Losses"] += 1
            else:
                scenario_records[t1]["Losses"] += 1
                scenario_records[t2]["Wins"] += 1
        
        # Determine status of each team
        for team in teams:
            wins = scenario_records[team]["Wins"]
            losses = scenario_records[team]["Losses"]
            if wins < 3 and losses < 3:
                round5_counts[team] += 1
    
    # Calculate probabilities
    probabilities = {team: count / total_outcomes for team, count in round5_counts.items()}
    
    # Prepare the result DataFrame
    prob_df = pd.DataFrame(list(probabilities.items()), columns=["Team", "Prob_Round5"])
    prob_df = prob_df.sort_values(by="Prob_Round5", ascending=False).reset_index(drop=True)
    
    return prob_df
